# Learning linkset correctness

**Research Questions**: 
- can we train a neural network distingushing between correct and uncorrect exactMatch?



## Preparing the training sets
The training set is based on a set of linksets that have been generated building [Linked Thesaurus fRamework for Environment (LusTRE)](http://linkeddata.ge.imati.cnr.it/) as part of the research activity carried out during two EU funded projects: NatureSDIPlus and eENVplus. 


The procedure adopted to prepare a view of the linksets with the label, BT,NT,RT are described in 
* [Preparing Linkset involving local dumps](http://localhost:8888/notebooks/ai-related/LinkCorrectess/PreparingLinksetWithLocalDumps.ipynb)
, which include all the linksets not involving DBPEDIA
* [Preparing Linkset involving Dbpedia](http://localhost:8888/notebooks/ai-related/LinkCorrectess/PreparingLinksetWithDBPEDIA.ipynb)

### Useful tutorial 
 -  A  useful tutorial about pandas's dataframe is available at https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/
 - creating and editing https://www.shanelynn.ie/using-pandas-dataframe-creating-editing-viewing-data-in-python/
 - [Advanced Jupyter Notebook Tricks — Part I](https://blog.dominodatalab.com/lesser-known-ways-of-using-notebooks/)

### Global variable 

In [25]:
### Global variables 
path="data/" # path where to find data
#namesa=['sBT','sprefLabel','sURI','oURI','oprefLabel','oBT', 'KindOfLink'] #column names for training data frame

### Generating the List of validated linksets

In [79]:
# https://dzone.com/articles/listing-a-directory-with-python
import os

TrainingFiles= filter(lambda x: x.endswith('EnrichedLinkeset.csv'),  os.listdir(path)) 
 
for file in TrainingFiles : 
    print(file)
       
    

Thist2AGROVOCEnrichedLinkeset.csv
Thist2EUROVOCEnrichedLinkeset.csv
ThIST2BpediaEnrichedLinkeset.csv
Thist2GEMETEnrichedLinkeset.csv


## What features are we going to consider to characterize a link?


Text and Conceptual similarity among prefered labels  and  broader terms are considered as significant features on which classify a link.

Different approaches are available in order to work out the text similarity

### word2Vec
- A pretrained model for text similarity http://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/ (**pretrained model in Users/bubu/model/ but instructions outdated** )
- example of usage in  https://radimrehurek.com/gensim/models/keyedvectors.html
Others  resources 
- https://medium.freecodecamp.org/how-to-get-started-with-word2vec-and-then-how-to-make-it-work-d0a2fca9dad3
- https://www.slideshare.net/lechatpito
- https://code.google.com/archive/p/word2vec/
- [Vector Representations of Words IN TF](https://www.tensorflow.org/tutorials/representation/word2vec)
- [Stanford courser - Word Vector Representations: word2vec](https://www.youtube.com/watch?v=ERibwqs9p38)
- using word2Vec in rapidMiner https://community.rapidminer.com/discussion/43860/synonym-detection-with-word2vec
 -https://www.neuralmarkettrends.com/word2vec-example-process-rapidminer

### Glo Ve

- https://medium.com/@japneet121/word-vectorization-using-glove-76919685ee0b

### Text Similarity
- Basic text similarities https://pypi.org/project/textdistance/



# A - Attempt 1: Let's initialize the Word2Vec with a pre-existing model

## Design choices
- **design choice 1**: We use the Google’s pre-trained model see [here](http://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/). It’s 1.5GB! It includes word vectors for a vocabulary of 3 million words and phrases that they trained on roughly 100 billion words from a Google News dataset. The vector length is 300 features.
- **design choice 2**: Similarity(s1,s2) implements a first attempt to work out the similarity between two set of words. It works out the max of sim on the pairs taken in the cardinal product of the sets, not considering the stoplist. 





In [29]:
#It takes very long to be executed
# https://radimrehurek.com/gensim/models/keyedvectors.html
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

#model = Word2Vec(common_texts, size=100, window=5, min_count=1, workers=4)
#word_vectors = model.wv
word_vectors = KeyedVectors.load_word2vec_format("/Users/bubu/model/GoogleNews-vectors-negative300.bin", binary=True)  # C bin format
 

## A1 - How to call the similarity between vectors

In [30]:
#similarity = word_vectors.similarity('africa'.lower(), 'Countries in Africa'.lower().split())
#print(similarity)
docdistance=word_vectors.wmdistance('africa', 'Africa')
print(docdistance)

0.675937254097414


In [31]:
#v= ['woman','man', 'house', 'pippo']
def printifvector(v):
    for e in v:
        print(e +":")
        try:
        #vector = word_vectors.wv.word_vec( word_vectors.doesnt_match(e), use_norm=True)
            print(word_vectors.get_vector(e)) 
        except KeyError as ex:
            print('exception for ' +e)


## A2 - Procedure  work to out similarity on BT according to the first attempt 


In [32]:
# Attempt to work out similarity between two set of words
#lw1 and lw2 are two documents containing set of words
def similarityBetweenSetsSplitingInWords(s1,s2):
    ## remove common words and not indexed words and tokenize
    stoplist = set('for a of the and to in'.split())
    if (type(s1) is not str) or (type(s2) is not str):
        return 0.0
    # tokenize and removing |
    remove= lambda x :x.replace('|',"")
    lw1=list(map(remove, s1.lower().split()))
    lw2=list(map(remove,s2.lower().split()))
    
    strip= lambda x :x.strip()
    lw1=list(map(strip, lw1))
    lw2=list(map(strip, lw2))
    
    
    
    ## what words are indexed?
    lw1 =  [word for word in lw1 if word not in stoplist]
    lw2 =  [word for word in lw2 if word not in stoplist]
    
    print(lw1)
    print(lw2)
    amax=-1.0
    
    # if one of the sets is empty it returns 0
    if not ((len(lw1) == 0) or (len(lw2) == 0)): 
        for i in lw1 :  
            lmax=-1.0
            try:
                #test if i is indexed otherwise exception
                word_vectors.get_vector(i) 
            except KeyError as ex:
                print('exception for ' +i)
                continue
            for ii in lw2 :
                try:
                     #test if i is indexed otherwise exception
                    word_vectors.get_vector(ii)
                except KeyError as ex:
                    print('exception for ' +ii)
                    continue
                sim=word_vectors.similarity(i,ii)
                #print('sim(%s, %s) = %f' %(i,ii,sim) )   
                lmax = max(sim , lmax)
                amax+=lmax
                       
    return amax



In [33]:
print(similarityBetweenSetsSplitingInWords("africa", 'africa'))


['africa']
['africa']
0.0


## maxInSplitWords (M)
Given 
*  two sets of words $S_1$ and $S_2$
*  a similarity functions
*  $(x_i,y_j) \in S_1 \times S_2$
  
maxInSplitWords implements the following mathematical function

$\text{maxInSplitWords(x,y,sim)}=\text{MAX}_{i,j}(sim(x_i,y_j))$

In [34]:
# Attempt to work out similarity between two set of words
# lw1 and lw2 are two documents containing set of words
# function is the similarity function to apply
# it returns the maximun similarity comaring the set product
def maxInSplitWords(s1,s2, function):
    ## remove common words and not indexed words and tokenize
    stoplist = set('for a of the and to in'.split())
    if (type(s1) is not str) or (type(s2) is not str):
        return 0.0
    # tokenize and removing |
    remove= lambda x :x.replace('|',"")
    lw1=list(map(remove, s1.lower().split()))
    lw2=list(map(remove,s2.lower().split()))
    
    strip= lambda x :x.strip()
    lw1=list(map(strip, lw1))
    lw2=list(map(strip, lw2))
    
    
    ## what words are indexed?
    lw1 =  [word for word in lw1 if word not in stoplist]
    lw2 =  [word for word in lw2 if word not in stoplist]
    
    print(lw1)
    print(lw2)
    lmax=float('nan')
    firstTime = True
    # if one of the sets is empty it returns 0
    if not ((len(lw1) == 0) or (len(lw2) == 0)): 
        for i in lw1 :  
            for ii in lw2 :
                sim=function(i,ii)
                if ( math.isnan(lmax)) :
                    lmax=sim
                else:
                    lmax = max(sim , lmax)                     
    return lmax



## SummingMax (SM)

Given 
*  two sets of words $S_1$ and $S_2$
*  a similarity functions
*  $(x_i,y_j) \in S_1 \times S_2$
  
summingMax implements the following mathematical function


$\text{summingMax(x,y,sim)}=\sum_i{\text{MAX}_{i,j}(sim(x_i,y_j))}$

In [35]:
# Attempt to work out similarity between two set of words
# lw1 and lw2 are two documents containing set of words
# function is the similarity function to apply
import math
def summingMax(s1, s2, function):
    ## remove common words and not indexed words and tokenize
    stoplist = set('for a of the and to in'.split())
    if (type(s1) is not str) or (type(s2) is not str):
        return 0.0
    # tokenize and removing |
    remove= lambda x :x.replace('|',"")
    lw1=list(map(remove, s1.lower().split()))
    lw2=list(map(remove,s2.lower().split()))
    
    strip= lambda x :x.strip()
    lw1=list(map(strip, lw1))
    lw2=list(map(strip, lw2))
    
    
    
    ## what words are indexed?
    lw1 =  [word for word in lw1 if word not in stoplist]
    lw2 =  [word for word in lw2 if word not in stoplist]
    
    print(lw1)
    print(lw2)
   
    amax=float('nan')
    #firstTime = True
    # if one of the sets is empty it returns 0
    if not ((len(lw1) == 0) or (len(lw2) == 0)): 
        for i in lw1 :  
            lmax=float('nan')
            for ii in lw2 :
                sim=function(i,ii)
                if (math.isnan(lmax)) :
                    lmax=sim
                    #firstTime =False
                else:
                    lmax = max(sim , lmax)
            if (math.isnan(amax)):
                amax=lmax;
            else: 
                amax+=lmax;
    return amax



In [36]:
print(summingMax('', '', textdistance.hamming.normalized_similarity))


[]
[]
nan


In [37]:
a = 'africa'.lower().split()
b = 'countries in africa'.lower().split()
c ='countries in Europe'.lower().split()
d ='Italy'
similarity = word_vectors.wmdistance(a, b)
print("{:.4f}".format(similarity))
similarity = word_vectors.wmdistance(a, c)
print("{:.4f}".format(similarity))
similarity = word_vectors.wmdistance(b, c)
print("{:.4f}".format(similarity))
similarity = word_vectors.wmdistance(a, d)
print("{:.4f}".format(similarity))

#a='great africa'.lower().strip().split()
#b='Countries in Africa'.lower().strip().split()
#print(a)
#print(b)
#similarity = word_vectors.distances(a,b )
#print(similarity)
#docdistance=word_vectors.wmdistance('woman', 'man')
#print(docdistance)

2.6975
3.7890
1.0915
4.1048


In [73]:
import textdistance
#df  data frame
df=THIST2DBPEDIA_df.drop_duplicates()
##TEST
#df=THIST2DBPEDIA_df[THIST2DBPEDIA_df['sBT']=='']

df['BT_similaritySInW']=0.0 
df['BT_wmdistance']=df['BT_Mwmdistance']= df['BT_SMwmdistance']= 0.0
df['BT_nhammingSim']= df['BT_MnhammingSim']= df['BT_SMnhammingSim']= 0.0



l = range(1, len(df))
for i in l:
    if (df.sBT.iloc[i]!='') and (df.oBT.iloc[i]!=''):
        df['BT_similaritySInW'][i] =similarityBetweenSetsSplitingInWords(df.sBT[i], df.oBT[i])
        df['BT_wmdistance'][i]=word_vectors.wmdistance(df.sBT[i], df.oBT[i])
        df['BT_Mwmdistance'][i]= maxInSplitWords(df.sBT[i], df.oBT[i], word_vectors.wmdistance)
        df['BT_SMwmdistance'][i]= summingMax(df.sBT[i], df.oBT[i], word_vectors.wmdistance)
        df['BT_nhammingSim'][i]=textdistance.hamming.normalized_similarity(df.sBT[i], df.oBT[i])
        df['BT_MnhammingSim'][i]=maxInSplitWords(df.sBT[i], df.oBT[i],textdistance.hamming.normalized_similarity)
        df['BT_SMnhammingSim'][i] =summingMax(df.sBT[i], df.oBT[i],textdistance.hamming.normalized_similarity)


['africa']
['countries', 'africa', 'landlocked', 'countries', 'west', 'african', 'countries', 'wikipedia', 'categories', 'named', 'after', 'countries', 'states', 'territories', 'established', '1960', 'republics']
exception for 1960


/Users/bubu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/bubu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


['africa']
['countries', 'africa', 'landlocked', 'countries', 'west', 'african', 'countries', 'wikipedia', 'categories', 'named', 'after', 'countries', 'states', 'territories', 'established', '1960', 'republics']


/Users/bubu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bubu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bubu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


['africa']
['countries', 'africa', 'landlocked', 'countries', 'west', 'african', 'countries', 'wikipedia', 'categories', 'named', 'after', 'countries', 'states', 'territories', 'established', '1960', 'republics']
['africa']
['countries', 'africa', 'landlocked', 'countries', 'west', 'african', 'countries', 'wikipedia', 'categories', 'named', 'after', 'countries', 'states', 'territories', 'established', '1960', 'republics']


...

['trinidad', 'tobago']
['islands', 'trinidad', 'tobago', 'wikipedia', 'categories', 'named', 'after', 'islands']
['trinidad', 'tobago']
['islands', 'trinidad', 'tobago', 'wikipedia', 'categories', 'named', 'after', 'islands']
['tuscany', 'islands', 'europe', 'europa', 'island']
['wikipedia', 'categories', 'named', 'after', 'islands', 'province', 'livorno', 'islands', 'tuscany']
exception for tuscany
exception for livorno
exception for tuscany
exception for livorno
exception for tuscany
exception for livorno
exception for tuscany
exception for livorno
exception for tuscany
['tuscany', 'islands', 'europe', 'europa', 'island']
['wikipedia', 'categories', 'named', 'after', 'islands', 'province', 'livorno', 'islands', 'tuscany']
['tuscany', 'islands', 'europe', 'europa', 'island']
['wikipedia', 'categories', 'named', 'after', 'islands', 'province', 'livorno', 'islands', 'tuscany']
['tuscany', 'islands', 'europe', 'europa', 'island']
['wikipedia', 'categories', 'named', 'after', 'islands', '

['uzbekistan']
['cities', 'uzbekistan', 'wikipedia', 'categories', 'named', 'after', 'populated', 'places', 'uzbekistan', 'populated', 'places', 'tashkent', 'region', 'capitals', 'asia', 'wikipedia', 'categories', 'named', 'after', 'capitals']
['uzbekistan']
['cities', 'uzbekistan', 'wikipedia', 'categories', 'named', 'after', 'populated', 'places', 'uzbekistan', 'populated', 'places', 'tashkent', 'region', 'capitals', 'asia', 'wikipedia', 'categories', 'named', 'after', 'capitals']
['uzbekistan']
['cities', 'uzbekistan', 'wikipedia', 'categories', 'named', 'after', 'populated', 'places', 'uzbekistan', 'populated', 'places', 'tashkent', 'region', 'capitals', 'asia', 'wikipedia', 'categories', 'named', 'after', 'capitals']
['uzbekistan']
['cities', 'uzbekistan', 'wikipedia', 'categories', 'named', 'after', 'populated', 'places', 'uzbekistan', 'populated', 'places', 'tashkent', 'region', 'capitals', 'asia', 'wikipedia', 'categories', 'named', 'after', 'capitals']
['venezuela']
['wikipedi

['caribbean', 'region', 'greater', 'antilles', 'central', 'america', 'antilles', 'west', 'indies']
['island', 'countries', 'wikipedia', 'categories', 'named', 'after', 'islands', 'countries', 'caribbean', 'countries', 'north', 'america', 'member', 'states', 'commonwealth', 'nations', 'states', 'territories', 'established', '1962', 'english-speaking', 'countries', 'territories', 'wikipedia', 'categories', 'named', 'after', 'countries']
['caribbean', 'region', 'greater', 'antilles', 'central', 'america', 'antilles', 'west', 'indies']
['island', 'countries', 'wikipedia', 'categories', 'named', 'after', 'islands', 'countries', 'caribbean', 'countries', 'north', 'america', 'member', 'states', 'commonwealth', 'nations', 'states', 'territories', 'established', '1962', 'english-speaking', 'countries', 'territories', 'wikipedia', 'categories', 'named', 'after', 'countries']
['caribbean', 'region', 'greater', 'antilles', 'central', 'america', 'antilles', 'west', 'indies']
['island', 'countries',

['caribbean', 'region', 'lesser', 'antilles', 'central', 'america', 'antilles', 'west', 'indies']
['island', 'countries', 'countries', 'caribbean', 'countries', 'north', 'america', 'member', 'states', 'commonwealth', 'nations', 'english-speaking', 'countries', 'territories', 'wikipedia', 'categories', 'named', 'after', 'countries']
['caribbean', 'region', 'lesser', 'antilles', 'central', 'america', 'antilles', 'west', 'indies']
['island', 'countries', 'countries', 'caribbean', 'countries', 'north', 'america', 'member', 'states', 'commonwealth', 'nations', 'english-speaking', 'countries', 'territories', 'wikipedia', 'categories', 'named', 'after', 'countries']
['caribbean', 'region', 'lesser', 'antilles', 'central', 'america', 'antilles', 'west', 'indies']
['island', 'countries', 'countries', 'caribbean', 'countries', 'north', 'america', 'member', 'states', 'commonwealth', 'nations', 'english-speaking', 'countries', 'territories', 'wikipedia', 'categories', 'named', 'after', 'countries'

['greece']
['wikipedia', 'categories', 'named', 'after', 'islands', 'tourism', 'greece', 'wikipedia', 'categories', 'named', 'after', 'populated', 'places', 'greece', 'municipalities', 'ionian', 'islands', '(region)']
['greece']
['wikipedia', 'categories', 'named', 'after', 'islands', 'tourism', 'greece', 'wikipedia', 'categories', 'named', 'after', 'populated', 'places', 'greece', 'municipalities', 'ionian', 'islands', '(region)']
['greece']
['wikipedia', 'categories', 'named', 'after', 'islands', 'tourism', 'greece', 'wikipedia', 'categories', 'named', 'after', 'populated', 'places', 'greece', 'municipalities', 'ionian', 'islands', '(region)']
['greece']
['wikipedia', 'categories', 'named', 'after', 'islands', 'tourism', 'greece', 'wikipedia', 'categories', 'named', 'after', 'populated', 'places', 'greece', 'municipalities', 'ionian', 'islands', '(region)']


#### Printing the similaritities according to the Similarity 

In [71]:
#df['KindOfLink']=df['KindOfLink'].fillna('W')
df.head()


,Unnamed: 0,s,o,KindOfLink,sBT,sNT,sRT,sPrefLabel,sAltLabels,oPrefLabel,...,oBT,oNT,oRT,similaritySInW,wmdistance,Mwmdistance,SMwmdistance,nhammingSim,MnhammingSim,SMnhammingSim
559,559,http://linkeddata.ge.imati.cnr.it/resource/ThI...,http://dbpedia.org/resource/Category:Seams,C,,,mineral deposits| mining| stratiform deposits|...,seams,,Seams,...,Sewing,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
560,560,http://linkeddata.ge.imati.cnr.it/resource/ThI...,http://dbpedia.org/resource/Category:Tides,E,,,ocean circulation| tidal currents| littoral er...,tides,,Tides,...,Geophysics| Moon| Physical oceanography| Tidal...,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
561,561,http://linkeddata.ge.imati.cnr.it/resource/ThI...,http://dbpedia.org/resource/Category:Tides,C,,,ocean circulation| tidal currents| littoral er...,tides,,Tides,...,Geophysics| Moon| Physical oceanography| Tidal...,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
562,562,http://linkeddata.ge.imati.cnr.it/resource/ThI...,http://dbpedia.org/resource/Category:Transport...,E,,,subways| pipelines| sediment transport,transportation,,Transportation,...,,,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
563,563,http://linkeddata.ge.imati.cnr.it/resource/ThI...,http://dbpedia.org/resource/Category:Ornamenta...,W,,,paleontology| shells,ornamentation,,Ornamentation,...,Melody| Musical terminology| Musical technique...,,Ornament,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:

#saving the results in Thist2DBPEDIA_ok_res.csv
df.to_csv(path+'Thist2DBPEDIA_ok_res.csv', sep=';') 

In [76]:
df['RT_similaritySInW']=0.0 
df['RT_wmdistance']=df['RT_Mwmdistance']= df['RT_SMwmdistance']= 0.0
df['RT_nhammingSim']= df['RT_MnhammingSim']= df['RT_SMnhammingSim']= 0.0

l = range(1, len(df))
for i in l:
    if (df.sRT.iloc[i]!='') and (df.oRT.iloc[i]!=''):
         df['RT_similaritySInW'][i] =similarityBetweenSetsSplitingInWords(df.sRT[i], df.oRT[i])
         df['RT_wmdistance'][i]=word_vectors.wmdistance(df.sRT[i], df.oRT[i])
         df['RT_Mwmdistance'][i]= maxInSplitWords(df.sRT[i], df.oRT[i], word_vectors.wmdistance)
         df['RT_SMwmdistance'][i]= summingMax(df.sRT[i], df.oRT[i], word_vectors.wmdistance)
         df['RT_nhammingSim'][i]=textdistance.hamming.normalized_similarity(df.sRT[i], df.oRT[i])
         df['RT_MnhammingSim'][i]=maxInSplitWords(df.sRT[i], df.oRT[i],textdistance.hamming.normalized_similarity)
         df['RT_SMnhammingSim'][i] =summingMax(df.sRT[i], df.oRT[i],textdistance.hamming.normalized_similarity)

['gulf', 'guinea', 'volta', 'river', 'volta', 'basin']
['guinea-bissau', 'equatorial', 'guinea', 'french', 'guiana', 'guyana', 'papua', 'new', 'guinea', 'new', 'guinea', 'guinea']
exception for guinea-bissau
exception for guiana
exception for guyana
exception for papua
exception for guinea-bissau
exception for guiana
exception for guyana
exception for papua
exception for guinea-bissau
exception for guiana
exception for guyana
exception for papua
exception for guinea-bissau
exception for guiana
exception for guyana
exception for papua
exception for guinea-bissau
exception for guiana
exception for guyana
exception for papua
exception for guinea-bissau
exception for guiana
exception for guyana
exception for papua


/Users/bubu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/bubu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


['gulf', 'guinea', 'volta', 'river', 'volta', 'basin']
['guinea-bissau', 'equatorial', 'guinea', 'french', 'guiana', 'guyana', 'papua', 'new', 'guinea', 'new', 'guinea', 'guinea']


/Users/bubu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/bubu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/bubu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] =

['gulf', 'guinea', 'volta', 'river', 'volta', 'basin']
['guinea-bissau', 'equatorial', 'guinea', 'french', 'guiana', 'guyana', 'papua', 'new', 'guinea', 'new', 'guinea', 'guinea']
['gulf', 'guinea', 'volta', 'river', 'volta', 'basin']
['guinea-bissau', 'equatorial', 'guinea', 'french', 'guiana', 'guyana', 'papua', 'new', 'guinea', 'new', 'guinea', 'guinea']
['gulf', 'guinea', 'volta', 'river', 'volta', 'basin']
['guinea-bissau', 'equatorial', 'guinea', 'french', 'guiana', 'guyana', 'papua', 'new', 'guinea', 'new', 'guinea', 'guinea']
['caucasus', 'azov', 'sea', 'ussr', 'urals']
['soviet', 'union']
exception for caucasus
exception for azov
exception for urals
['caucasus', 'azov', 'sea', 'ussr', 'urals']
['soviet', 'union']
['caucasus', 'azov', 'sea', 'ussr', 'urals']
['soviet', 'union']
['caucasus', 'azov', 'sea', 'ussr', 'urals']
['soviet', 'union']
['caucasus', 'azov', 'sea', 'ussr', 'urals']
['soviet', 'union']
['malaysia', 'brunei', 'indonesia', 'pacific', 'ocean']
['kalimantan']
ex

...

['passband', 'filters', 'wiener', 'filters', 'continuous', 'filters', 'geotextiles', 'recursive', 'filters', 'seismic', 'methods', 'dispersive', 'filters', 'multichannel', 'methods', 'adaptive', 'filters', 'numerical', 'filters', 'multichannel', 'filters', 'kalman', 'filters', 'elastic', 'waves', 'spatial', 'frequency', 'filters', 'analog', 'filters', 'discrete', 'filters', 'optimal', 'filters', 'deconvolution', 'signals', 'noise', 'distortion', 'seismology']
['signal', 'processing', 'filter', 'spam', 'filtering', 'anti-spam', 'optical', 'filters']
['passband', 'filters', 'wiener', 'filters', 'continuous', 'filters', 'geotextiles', 'recursive', 'filters', 'seismic', 'methods', 'dispersive', 'filters', 'multichannel', 'methods', 'adaptive', 'filters', 'numerical', 'filters', 'multichannel', 'filters', 'kalman', 'filters', 'elastic', 'waves', 'spatial', 'frequency', 'filters', 'analog', 'filters', 'discrete', 'filters', 'optimal', 'filters', 'deconvolution', 'signals', 'noise', 'distorti

['sewage', 'sludge', 'thermal', 'pollution', 'leaking', 'underground', 'storage', 'tanks', 'sulfuric', 'acid', 'toxic', 'materials', 'urban', 'geology', 'cyanides', 'water', 'treatment', 'metals', 'regulations', 'surface', 'water', 'waste', 'disposal', 'potability', 'hydrology', 'heavy', 'metals', 'urban', 'environment', 'pollutants', 'polluted', 'water', 'soil', 'gases', 'impurities', 'geochemistry', 'background', 'level', 'remediation', 'purification', 'environmental', 'monitoring', 'radioactive', 'waste', 'point', 'sources', 'industrial', 'waste', 'human', 'ecology', 'hydrocarbons', 'acid', 'mine', 'drainage', 'environmental', 'geology', 'medical', 'geology', 'waste', 'disposal', 'sites', 'sewers', 'geochemical', 'anomalies', 'conservation', 'oil', 'spills', 'geologic', 'hazards', 'dilution', 'decontamination', 'water', 'management', 'environmental', 'impact', 'evaluation', 'land', 'use', 'waste', 'water', 'reclamation', 'near-field', 'radioactivity', 'bioremediation', 'protected', 

['popular', 'geology', 'current', 'research']
['years', 'births', 'by', 'year', 'deaths', 'by', 'year', 'years', 'music', 'films', 'by', 'year', 'history', 'years', 'film']
['popular', 'geology', 'current', 'research']
['years', 'births', 'by', 'year', 'deaths', 'by', 'year', 'years', 'music', 'films', 'by', 'year', 'history', 'years', 'film']
['popular', 'geology', 'current', 'research']
['years', 'births', 'by', 'year', 'deaths', 'by', 'year', 'years', 'music', 'films', 'by', 'year', 'history', 'years', 'film']
['popular', 'geology', 'current', 'research']
['years', 'births', 'by', 'year', 'deaths', 'by', 'year', 'years', 'music', 'films', 'by', 'year', 'history', 'years', 'film']
['popular', 'geology', 'current', 'research']
['years', 'births', 'by', 'year', 'deaths', 'by', 'year', 'years', 'music', 'films', 'by', 'year', 'history', 'years', 'film']
['paleontology', 'predation', 'nutrients', 'trophic', 'analysis', 'diet', 'metabolism']
['food', 'science', 'digestive', 'system']
['pa

['crystal', 'structure']
['quasiregular', 'polyhedra', 'platonic', 'solids', 'archimedean', 'solids', 'images', 'polyhedra', 'tessellation', 'kepler–poinsot', 'polyhedra', 'prismatoid', 'polyhedra', 'uniform', 'polyhedra', 'catalan', 'solids', 'johnson', 'solids', 'pyramids', 'bipyramids']
['crystal', 'structure']
['quasiregular', 'polyhedra', 'platonic', 'solids', 'archimedean', 'solids', 'images', 'polyhedra', 'tessellation', 'kepler–poinsot', 'polyhedra', 'prismatoid', 'polyhedra', 'uniform', 'polyhedra', 'catalan', 'solids', 'johnson', 'solids', 'pyramids', 'bipyramids']
['runoff', 'atmospheric', 'precipitation', 'hydrology', 'crystallization', 'geochemistry', 'sedimentation', 'salt', 'water', 'watersheds', 'storms', 'water']
['clouds']
['runoff', 'atmospheric', 'precipitation', 'hydrology', 'crystallization', 'geochemistry', 'sedimentation', 'salt', 'water', 'watersheds', 'storms', 'water']
['clouds']
['runoff', 'atmospheric', 'precipitation', 'hydrology', 'crystallization', 'geoc

['bogs', 'marshes', 'lacustrine', 'features', 'shore', 'features', 'spartina', 'alterniflora', 'conservation', 'geomorphology', 'vegetation', 'wilderness', 'areas', 'grasslands', 'swamps', 'ecology', 'fluvial', 'features']
['landforms', 'bodies', 'water']
['bogs', 'marshes', 'lacustrine', 'features', 'shore', 'features', 'spartina', 'alterniflora', 'conservation', 'geomorphology', 'vegetation', 'wilderness', 'areas', 'grasslands', 'swamps', 'ecology', 'fluvial', 'features']
['landforms', 'bodies', 'water']
['autochthons', 'allochthons', 'structural', 'geology', 'tectonics', 'erosion', 'features', 'overthrust', 'faults']
['microsoft', 'windows']
exception for autochthons
exception for allochthons
['autochthons', 'allochthons', 'structural', 'geology', 'tectonics', 'erosion', 'features', 'overthrust', 'faults']
['microsoft', 'windows']
['autochthons', 'allochthons', 'structural', 'geology', 'tectonics', 'erosion', 'features', 'overthrust', 'faults']
['microsoft', 'windows']
['autochthons

In [77]:

#saving the results in Thist2DBPEDIA_ok_res.csv
df.to_csv(path+'Thist2DBPEDIA_ok_resWithRT.csv', sep=';') 

# Learning the Model


https://academy.rapidminer.com/courses/creating-a-decision-tree-model